# Stock Prediction Training & Accuracy Evaluation

Use this notebook to train the lightweight LSTM forecaster defined in `app/services/stock_prediction_service.py`, evaluate its historical accuracy, and inspect fresh predictions.

## 1. Environment Setup
Make sure you start the notebook from the project root so imports succeed. GPU is disabled by default for reproducibility, but you can remove that line if you want to leverage CUDA.

In [7]:
import os
import sys
from pathlib import Path

def find_project_root(start: Path) -> Path:
    """Locate the project root by searching upwards for an `app/` package (or common root files)."""
    possible_root_files = ("setup.py", "pyproject.toml", "main.py", "requirements.txt")
    for p in [start] + list(start.parents):
        if (p / "app").is_dir():
            return p
        if any((p / f).is_file() for f in possible_root_files):
            return p
    return start

PROJECT_ROOT = find_project_root(Path.cwd().resolve())
sys.path.insert(0, str(PROJECT_ROOT))

# Disable GPU for consistent results unless you explicitly want to use CUDA
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "")
PROJECT_ROOT

PosixPath('/home/khaitran/project/Azure-OpenAI_StockTool')

In [8]:
# Quick import sanity check to ensure `app` is importable in this notebook environment
try:
    import app
    print("app imported OK")
    from app.services.stock_prediction_service import train_model, evaluate_model_accuracy, predict_stock_price
    print("stock_prediction_service imports OK")
except Exception as e:
    print("ImportError:", type(e).__name__, str(e))

app imported OK
stock_prediction_service imports OK


## 2. Configuration
Adjust the ticker symbol, training period, evaluation window, and forecast horizon as needed.

In [9]:
SYMBOL = 'AAPL'
TRAIN_PERIOD = '2y'   # Historical window for fitting the model
EVAL_PERIOD = '6mo'    # Historical window used for accuracy verification
FORECAST_DAYS = 7      # Number of future trading days to predict

## 3. Train / Refresh Model
This reuses the `train_model` helper so the scaler and weights are saved under `models/stock_predictions/`.

In [10]:
from app.services.stock_prediction_service import train_model

train_result = train_model(symbol=SYMBOL, period=TRAIN_PERIOD, save_model=True)
train_result

/home/khaitran/project/Azure-OpenAI_StockTool/app/services/stock_prediction_service.py:78: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["date"] = pd.to_datetime(df["date"])
/home/khaitran/project/Azure-OpenAI_StockTool/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


{'symbol': 'AAPL',
 'status': 'success',
 'model_path': 'models/stock_predictions/AAPL_model.keras',
 'scaler_path': 'models/stock_predictions/AAPL_scaler.pkl',
 'metrics': {'train_loss': 0.009134729392826557,
  'val_loss': 0.0038976334035396576,
  'train_mae': 0.06849193572998047,
  'val_mae': 0.043352171778678894,
  'train_mae_return': 0.012586706915722626,
  'val_mae_return': 0.010880526183343252,
  'train_rmse_return': 0.019337220774912175,
  'val_rmse_return': 0.015294356142852638,
  'train_mape_pct': 39155.835957055184,
  'val_mape_pct': 179.55170239689133},
 'data_info': {'train_samples': 351,
  'test_samples': 88,
  'features': ['return'],
  'lookback_days': 60},
 'source': 'lstm_prediction'}

## 4. Evaluate Historical Accuracy
Use the new `evaluate_model_accuracy` helper to replay recent history and compute MAE/RMSE/MAPE in dollars.

In [11]:
from app.services.stock_prediction_service import evaluate_model_accuracy

accuracy_report = evaluate_model_accuracy(SYMBOL, period=EVAL_PERIOD)
accuracy_report

/home/khaitran/project/Azure-OpenAI_StockTool/app/services/stock_prediction_service.py:78: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["date"] = pd.to_datetime(df["date"])


{'symbol': 'AAPL',
 'period': '6mo',
 'samples': 66,
 'metrics': {'mae_return': 0.010514292330097772,
  'rmse_return': 0.014444543511229285,
  'mape_pct': 180.83337351436043},
 'recent_examples': [{'actual_return_pct': 2.16,
   'predicted_return_pct': 0.39,
   'error_pct': -1.77},
  {'actual_return_pct': -0.65,
   'predicted_return_pct': 0.39,
   'error_pct': 1.03},
  {'actual_return_pct': -0.19,
   'predicted_return_pct': 0.38,
   'error_pct': 0.57},
  {'actual_return_pct': -0.2, 'predicted_return_pct': 0.38, 'error_pct': 0.57},
  {'actual_return_pct': -1.82,
   'predicted_return_pct': 0.37,
   'error_pct': 2.19}],
 'source': 'lstm_prediction'}

## 5. Generate Fresh Forecasts
Finally, generate the forward-looking path so you can compare it with the metrics above.

In [12]:
from app.services.stock_prediction_service import predict_stock_price
import pandas as pd

prediction = predict_stock_price(SYMBOL, days=FORECAST_DAYS, auto_train=False)
display(prediction['summary'])
pd.DataFrame(prediction['predictions'])

/home/khaitran/project/Azure-OpenAI_StockTool/app/services/stock_prediction_service.py:399: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["date"] = pd.to_datetime(df["date"])


{'final_predicted_price': 273.26,
 'total_change': 5.8,
 'total_change_pct': 2.17,
 'trend': '上昇',
 'trend_en': 'bullish'}

,date,predicted_price,change_from_current,change_pct,predicted_return_pct
0,2025-11-18,268.40,0.94,0.35,0.35
1,2025-11-19,269.30,1.84,0.69,0.33
2,2025-11-20,270.15,2.69,1.01,0.32
3,2025-11-21,270.96,3.50,1.31,0.30
4,2025-11-24,271.75,4.29,1.60,0.29
5,2025-11-25,272.51,5.05,1.89,0.28
6,2025-11-26,273.26,5.80,2.17,0.28
